## SQLite

Embora a biblioteca Pandas seja a indica para o tratamento da maior parte das situações, por vezes torna-se necessário a utilização de bases de dados e das operações associadas às bases de dados.

Um dos motores de bases de dados disponíveis para pequenas implementações é o SQLite, cuja utilização se mostra de seguida.

Há operações que tipicamente são mais rápidas com o SQLite (select, filter e order) e outras que são mais rápidas com o Pandas (group by, load e join).

INSERT INTO empregado VALUES (kVcqvncd,f,1951,17632.84458925844);
INSERT INTO empregado VALUES (lAkHndJk,e,1968,26757.41549837873);
INSERT INTO empregado VALUES (knktcWME,d,1988,58920.147713077364);
INSERT INTO empregado VALUES (UanlXumG,e,1969,43956.01899492496);
INSERT INTO empregado VALUES (bXtQwjRc,d,1942,12770.075573823411);
INSERT INTO empregado VALUES (qDsYPodg,g,1961,85343.50054203243);
INSERT INTO empregado VALUES (tAZFCAzX,d,1946,74982.4984803583);
INSERT INTO empregado VALUES (mgbBLBOf,a,1936,33463.752116623065);


In [1]:
import sqlite3
conn = sqlite3.connect('empregados.db')

c = conn.cursor()
#Comando para criar uma tabela de dados (ex. empregado)
c.execute('CREATE TABLE empregado (nome varchar(255), departamento char(1), ano_nascimento int, salario double);')
#Criacao da tabela bonus
c.execute('CREATE TABLE bonus (nome varchar(255), bonus double);')

#Depois de criar as tabelas não volte a correr o código acima
#Caso deseje recriar a BD, ou apaga o ficheiro da mesma, empregados.db, ou faz DROP das tabelas desejadas.

In [7]:
#Criacao de dados random para preencher a tabela empregado (populate)
#Existem bibliotecas em Python que permitem preenchimento das tabelas com dados mais parecidos com os reais ex. https://faker.readthedocs.io/en/master/#
import random
import string

def random_string(length, chars=string.ascii_letters):
    return ''.join(random.choice(chars) for _ in range(length))

def gerar_empregados(n):
    for _ in range(n):
        yield (random_string(8), random_string(1, chars='abcdefg'), random.randint(1900, 2000), random.uniform(1e4, 1e5))

#Teste para ver se gera as linahs corretamente
for linha in gerar_empregados(3):
    linha2 = "INSERT INTO empregado VALUES ('" + linha[0] + "','" + linha[1] + "'," + str(int(linha[2])) + "," + str(linha[3])+");" 
    print(linha2)

INSERT INTO empregado VALUES ('JiVapfJZ','c',1931,91068.19509794368);
INSERT INTO empregado VALUES ('FwkFQQDM','d',1996,23784.2329737553);
INSERT INTO empregado VALUES ('sMtJWaiK','b',1961,37393.51576237766);


In [4]:
#Insercao de 10000 linhas utilizando o código acima
#Pode alterar o valor 10000 para o valor que desejar quanto estiver a fazer testes à BD

for lin in gerar_empregados(10000):
    linha = "INSERT INTO empregado (nome, departamento, ano_nascimento, salario) VALUES ('" + lin[0] + "','" + lin[1] + "'," + str(lin[2]) + "," + str(lin[3])+");"
    c.execute(linha)

for row in c.execute('SELECT nome FROM empregado'):
    c.execute('INSERT INTO bonus (nome, bonus) VALUES ("' + row[0] + '",' + str(random.uniform(1e4, 1e5)) + ')')

#Guardar as alterações
conn.commit()

#Fechar a ligação à BD
conn.close()

De seguida vai-se testar o desempenho de execução da leitura de duas colunas à tabela empregado, uma utilizando o motor de BD SQLite e outra utilizando o Pandas (Fonte: http://bit.ly/2G93lbj). 

In [6]:
#Para listar os valores das tabelas utilizaos seguintes métodos / funções:
import sqlite3
import pandas as pd
from timeit import timeit

conn = sqlite3.connect('empregados.db')
df_empregado = pd.read_sql_query('SELECT * FROM empregado', conn)
print(df_empregado.head())
    
def sql_select(conn):
    conn.execute('SELECT nome, departamento FROM empregado')
    conn.commit()

def pd_select(df_empregado):
    df_empregado[["nome", "departamento"]]
        
%timeit sql_select(conn)
%timeit pd_select(df_empregado)

conn.close()

print("Comente os valores que obteve.")

       nome departamento  ano_nascimento       salario
0  dmpsFlxc            e            1976  45376.294488
1  DZHqDfDo            f            1952  27786.967086
2  hGhAPxvC            c            1952  72989.332440
3  XFVldhgd            b            1991  23090.118032
4  xpuTWYpx            g            1910  61787.448490
166 µs ± 2.47 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1.4 ms ± 183 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Exercício 1:
    
Teste as restantes situações de manipulações de dados, com as seguintes funções:

In [8]:
def sql_sort(c):
    c.execute('SELECT * FROM empregado ORDER BY nome ASC;')

def pd_sort(df):
    df.sort_values(by='nome')

def sql_join(c):
    c.execute('SELECT empregado.nome, empregado.salario + bonus.bonus FROM empregado INNER JOIN bonus ON empregado.nome = bonus.nome')

def pd_join(df_emp, df_bonus):
    joined = df_emp.merge(df_bonus, on='nome')
    joined['total'] = joined['bonus'] + joined['salario']

def sql_filtrar(c):
    c.execute('SELECT * FROM empregado WHERE departamento = "a";')

def pd_filtrar(df):
    df[df['departamento'] == 'a']

def sql_groupby(c):
    c.execute('SELECT avg(ano_nascimento), sum(salario) FROM empregado GROUP BY departamento;')

def pd_groupby(self):
    df.groupby("departamento").agg({'ano_nascimento': np.mean, 'salario': np.sum})

## Exercício 2:

Compare a performance do SQLite vs Pandas utilizando um DataSet com 100 000 linhas.

Utilize o dataset relativo a vinhos disponível em https://www.kaggle.com/zynicide/wine-reviews/version/4

Olhe para os dados e faça medições de tempo relativas a alguns cálculos (ex. preco médio do vinho, vinhos mais baratos com a melhor classificação, etc.).

## Sintaxe Pandas vs SQL

No seguinte link http://bit.ly/2G9skev podem ver a comparação entre a sintaxe da biblioteca Pandas e a linguagem SQL. 

## SQLAlchemy

Para realizar operações mais complexas consulte a seguinte biblioteca https://www.sqlalchemy.org/

## PostgreSQL

A base de dados SQLite permite guardar informação e realizar um grande conjunto de tarefas sendo uma das preferidas pelos analistas de dados.

No entanto, se desejar trabalhar com bases de dados mais robusta sugere-se a utilização das base de dados PostgreSQL.

Para poder interagir com este motor de base de dados terá primeiramente que instalar o PostGreSQL na sua máquina.

Faça import da bilioteca psycopg2 para trabalhar com o PostgreSQL.

```
import psycopg2
```

Se der erro terá de instalar o package com o comando: 

```
pip install psycopg2
```

Veja um tutorial de como utilizar a biblioteca psycopg2.